In [39]:
from agents import Agent, Runner, trace, function_tool
import sendgrid
from sendgrid.helpers.mail import Mail, To, From, Content, Email
from dotenv import load_dotenv
load_dotenv(override=True)
import asyncio
import os

In [40]:
joker_agent = Agent(name = "Jokester Agent", instructions="You are a joker, you tell jokes of given topic", model = 'gpt-4o-mini')

In [41]:
message = "Give a joke on people who drink coffee"

response = await Runner.run(joker_agent, message)

In [42]:
print(response.final_output)

Why do coffee lovers always seem so positive?  

Because they know how to espresso themselves!


In [43]:
# Create agents - 1,2,3 (All writes cold emails)
# put them in a single list 
# Convert to string 
# Create new Agent -> pass string ans ask to select best email 
# create agent to send the best selected email 

In [44]:
instruction1 = "You are a an agent working for an organisation that does consulting.\
    a company that provides consulting to the firms powered by AI.\
    You write professional, serious cold emails"

instruction2 = "You are a an agent working for an organisation that does consulting.\
    a company that provides consulting to the firms powered by AI.\
    You write witty, engaging serious cold emails"

instruction3 = "You are a an agent working for an organisation that does consulting.\
    a company that provides consulting to the firms powered by AI.\
    You write concise, to the point cold emails"

In [45]:
agent1 = Agent(
    name = "Professional email Agent",
    instructions=instruction1,
    model = 'gpt-4o-mini'
)

agent2 = Agent(
    name = "Engaging email agent",
    instructions=instruction2,
    model = 'gpt-4o-mini'
)

agent3 = Agent(
    name = "Busy email agent",
    instructions=instruction3,
    model = 'gpt-4o-mini'
)

In [46]:
with trace("email agent1"):
    result = await Runner.run(agent1, input = "Write a cold email")
    print(result.final_output)

Subject: Unlocking AI Potential for Your Business

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I am with [Your Company], a consulting firm specializing in AI-driven solutions that empower organizations to optimize operations and enhance decision-making.

In today’s rapidly evolving landscape, leveraging Artificial Intelligence is no longer optional but fundamental for maintaining a competitive edge. We have collaborated with numerous firms in your industry, helping them successfully integrate AI strategies to achieve measurable results.

I would love the opportunity to discuss how our expertise could support [Recipient's Company] in harnessing the full potential of AI. Whether it’s streamlining processes, enhancing customer engagement, or driving innovation, we’re here to assist every step of the way.

Are you available for a brief call next week? I look forward to the possibility of working together.

Best regards,

[Your Name]  
[Your Pos

In [47]:
with trace("email agent3"):
    result = await Runner.run(agent3, input = "Write a cold email")
    print(result.final_output)

Subject: Unlock Your Firm's Potential with AI Consulting

Hi [Recipient's Name],

I hope this message finds you well. I'm [Your Name] from [Your Company Name], specializing in AI-driven consulting tailored for firms like yours.

As businesses increasingly harness AI, we help organizations streamline processes, enhance decision-making, and drive growth. I believe we can assist [Recipient's Company] in unlocking new efficiencies and strategies.

Could we schedule a brief call next week to discuss how we can support your goals?

Looking forward to your reply.

Best,  
[Your Name]  
[Your Position]  
[Your Company Name]  
[Your Phone Number]  
[Your Website]  


Getting output of all the agents

In [48]:
message = "Write a cold Sales email"

with trace("Parallel Cold emails"):
    results = await asyncio.gather(
        Runner.run(agent1, message),
        Runner.run(agent2, message),
        Runner.run(agent3, message)
    )

print(results)

[RunResult(input='Write a cold Sales email', new_items=[MessageOutputItem(agent=Agent(name='Professional email Agent', instructions='You are a an agent working for an organisation that does consulting.    a company that provides consulting to the firms powered by AI.    You write professional, serious cold emails', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), raw_item=ResponseOutputMessage(id='msg_685eb10999748198a277c6c33a7a777d01c21060f1249157', content=[ResponseOutputTex

In [49]:
for result in results:
    print(result.final_output)

Subject: Unlock the Full Potential of AI in Your Business

Dear [Recipient's Name],

I hope this message finds you well. I am reaching out to introduce you to [Your Company Name], where we specialize in helping businesses like yours leverage artificial intelligence to drive efficiency, innovation, and growth.

As you may know, the landscape of AI is rapidly evolving, and organizations that harness its power effectively stand to gain a significant competitive edge. However, integrating AI into your operations can be complex and challenging. That's where we come in.

At [Your Company Name], our team of AI consultants has a proven track record of delivering tailored strategies that enhance productivity and optimize processes. Whether you’re looking to implement machine learning solutions, improve data management, or enhance customer experience through AI-driven insights, we can help.

I would love the opportunity to discuss how we can support [Recipient's Company Name] in achieving its AI

In [50]:
best_email_picker = Agent(
    name = "Email Picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model = "gpt-4o-mini"
)

In [51]:
message = "Write cold consulting email"

with trace("Selecting best email"):
    results = await asyncio.gather(
        Runner.run(agent1, message),
        Runner.run(agent2, message),
        Runner.run(agent3, message),
    )

    outputs = [result.final_output for result in results]
    emails = "Cold Consulting Emails\n\n".join(outputs)

    best = await Runner.run(best_email_picker, emails)
    print(f"Best email: {best.final_output}")

Best email: Subject: Unlock the Future of Your Business with AI Insights 🚀

Hi [Recipient's Name],

I hope this email finds you thriving in today’s fast-paced digital landscape! As a fellow explorer of innovation, I wanted to reach out and share some thoughts on how we can transform your firm's potential with the power of AI.

Imagine having a virtual assistant that doesn’t just manage tasks but intelligently analyzes data, predicts trends, and unlocks hidden opportunities within your operations. Sounds like a plot twist from a sci-fi novel, right? Well, it’s the reality we help our clients achieve every day.

At [Your Company Name], we specialize in guiding businesses like yours through the AI revolution. Whether you're looking to enhance decision-making processes or automate routine tasks, our consulting services are tailored to your unique needs.

Let’s not let the future happen to us—let's shape it together! I’d love to schedule a quick chat to explore how we can elevate your firm 

Making use of the tools

In [52]:
sales_agent1 = Agent(
    name = "Professional Sales Agent",
    instructions= instruction1,
    model = "gpt-4o-mini"
)
sales_agent2 = Agent(
    name = "Engaging Sales Agent",
    instructions= instruction2,
    model = "gpt-4o-mini"
)
sales_agent3 = Agent(
    name = "Busy Sales Agent",
    instructions= instruction3,
    model = "gpt-4o-mini"
)

In [53]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all consulting prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("uditsdaily@gmail.com")  # Change to your verified sender
    to_email = To("uditdgreat@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Want to take your organisation to next level?", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}


In [54]:
description = "Write a cold Consulting email"

tool1 = agent1.as_tool(tool_name = "write_email1", tool_description=description)
tool2 = agent2.as_tool(tool_name = "write_email2", tool_description=description)
tool3 = agent3.as_tool(tool_name = "write_email3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='write_email1', description='Write a cold Consulting email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'write_email1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002607143BE20>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='write_email2', description='Write a cold Consulting email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'write_email2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000026071439C60>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='write_email3', description='Write a cold Consulting email', params_json_schema={'properties': {'input': {'title': 'Input', '

In [55]:
instructions ="You are a sales manager working for ConsultAI. You use the tools given to you to generate cold consulting emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 write_email tools once before choosing the best one. \
You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user."

email_sender = Agent(
    name = "Manager",
    instructions=instructions,
    model = "gpt-4o-mini",
    tools = tools
)

message = "Send a Cold email addressed to Dear CEO from Udit"

with trace("Email Try"):
    response = await Runner.run(email_sender, message)